In [1]:
import pandas as pd
import yfinance as yf
import numpy as np

In [2]:
nifty_500 = pd.read_csv('ind_nifty500list.csv')

In [3]:
nifty_500['Symbol_Yahoo'] = nifty_500['Symbol'].apply(lambda x : '{}.{}'.format(x,'NS')) 

In [4]:
nifty_500.head()

,Company Name,Industry,Symbol,Series,ISIN Code,Symbol_Yahoo
0,3M India Ltd.,SERVICES,3MINDIA,EQ,INE470A01017,3MINDIA.NS
1,ACC Ltd.,CEMENT & CEMENT PRODUCTS,ACC,EQ,INE012A01025,ACC.NS
2,AIA Engineering Ltd.,INDUSTRIAL MANUFACTURING,AIAENG,EQ,INE212H01026,AIAENG.NS
3,APL Apollo Tubes Ltd.,METALS,APLAPOLLO,EQ,INE702C01019,APLAPOLLO.NS
4,AU Small Finance Bank Ltd.,FINANCIAL SERVICES,AUBANK,EQ,INE949L01017,AUBANK.NS


In [5]:
Industries = list(nifty_500['Industry'].unique())
Industry_code = [i for i in range(1,len(Industries) + 1)]
Industry_dict = dict(zip(Industries,Industry_code))

In [6]:
start = '2015-05-15'
end = '2020-05-16'

In [82]:
z = yf.Ticker("ITC.NS")

In [84]:
z.info

{'zip': '700071',
 'sector': 'Consumer Defensive',
 'fullTimeEmployees': 33305,
 'longBusinessSummary': 'ITC Limited engages in the fast moving consumer goods, hotels, paperboards, paper and packaging, agri, and information technology businesses worldwide. The company primarily offers cigarettes and cigars; staples, spices, biscuits, confectionery and gums, snacks, noodles and pasta, beverages, dairy, ready to eat meals, chocolate, coffee, and frozen foods; personal care products; education and stationery products; safety matches; and incense sticks under various brands. It also retails formals and casual wear products, and other lifestyle products through the Wills Lifestyle stores. In addition, the company offers paper boards and specialty paper products; and packaging products, such as carton board, flexible, and tobacco packaging products, as well as operates approximately 100 hotels under the ITC Hotel, WelcomHotel, Fortune, and WelcomHeritage brands. Further, it exports feed ingr

In [8]:
z = z.history(start = start,end = end)

In [18]:
list(z.loc['2015-11-10'].values)

[209.65, 212.26, 206.17, 207.16, 7228746.0, 0.0, 0.0]

In [19]:
list(z.loc['2019-02-12'].values)

[271.84, 275.23, 269.44, 270.13, 7341352.0, 0.0, 0.0]

In [20]:
list(z.loc['2019-03-28'].values)

[288.57, 296.32, 287.49, 294.36, 17752380.0, 0.0, 0.0]

In [25]:
z.shape

(1230, 7)

In [64]:
index = [pd.Timestamp('2015-11-11'),pd.Timestamp('2019-02-13'),pd.Timestamp('2019-03-29')]

In [73]:
sectors = []

for symbol in nifty_500['Symbol_Yahoo'].values :
    
    ticker = yf.Ticker(symbol)
    df = ticker.history(start = start,end = end,actions = False)
    
    df.columns = ['open','high','low','close','volume']
    
    if df.shape[0] == z.shape[0] :
        
        temp = []
        
        sector = nifty_500[nifty_500['Symbol_Yahoo'] == symbol]['Industry'].values[0]
        sectors.append(Industry_dict[sector])
        
        temp.append(list(df.loc['2015-11-10'].values))
        temp.append(list(df.loc['2019-02-12'].values))
        temp.append(list(df.loc['2019-03-28'].values))
        temp_df = pd.DataFrame(temp,index = index,columns = df.columns)
        
        df = pd.concat([df,temp_df])
        
        df = df.sort_index(axis = 0) 
        
        df.to_csv('data/daily/{}.csv'.format(symbol))

sectors = np.array(sectors)        
np.save('data.npy',sectors)

In [67]:
df.tail()

,open,high,low,close,volume,dividend,split
2020-05-14,438.60,441.00,430.15,432.35,10607.0,0.0,0.0
2020-05-15,434.00,440.55,420.00,426.35,43179.0,0.0,0.0
2015-11-11,1301.55,1316.43,1268.48,1278.23,18739.0,0.0,0.0
2019-02-13,1021.44,1021.44,976.02,980.95,9443.0,0.0,0.0
2019-03-29,1133.38,1148.97,1130.50,1144.29,30525.0,0.0,0.0
